<a href="https://colab.research.google.com/github/manju2021/cxr_images/blob/master/fidscore_calculationv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# example of calculating the frechet inception distance in Keras for cifar10
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10

%matplotlib inline
import tensorflow as tf
from tensorflow import keras
import numpy as np
#import plot_utils
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython import display
print('Tensorflow version:', tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,BatchNormalization,Reshape,Conv2DTranspose,Conv2D,LeakyReLU,Dropout,Flatten,ReLU

Tensorflow version: 2.8.0


In [ ]:
#Code for mounting the google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#set the path to the main dir
import os
main_dir= '/content/drive/MyDrive/Colab Notebooks/cxr_v1'


In [ ]:
New_train_covid_names=os.listdir(main_dir)
print(New_train_covid_names[1:10])



['COVID-1003.png', 'COVID-1003 - Copy.png', 'COVID-1005.png', 'COVID-1009 - Copy.png', 'COVID-101 - Copy.png', 'COVID-1011.png', 'COVID-1014.png', 'COVID-1023.png', 'COVID-1025.png']


In [ ]:
#print the total no of images present in each dir
print("Total images present in the training set :",len(New_train_covid_names))
#

Total images present in the training set : 507


In [ ]:
##Keras model
real_dataset = keras.preprocessing.image_dataset_from_directory(
    main_dir, label_mode=None, image_size=(256,256), batch_size=64,shuffle=True
)
real_dataset = real_dataset.map(lambda x: x / 255.00)

Found 507 files belonging to 1 classes.


In [ ]:
10/2

5.0

In [ ]:
10/2.000

5.0

In [ ]:
#sys_images='/content/drive/MyDrive/Colab Notebooks/cxr_v1_exp4/200_epoch' 
#: Fid score : 324.89
#sys_images='/content/drive/MyDrive/Colab Notebooks/cxr_v1_exp3/200_epoch' : Fid score : 330.11

syn_images_exp2_v3='/content/drive/MyDrive/Colab Notebooks/cxr1_exp2_v3/1000_epoch' 
syn_images_exp4_v3='/content/drive/MyDrive/Colab Notebooks/cxr1_exp4_v3/1000_epoch'
syn_images_exp9_v3='/content/drive/MyDrive/Colab Notebooks/aug_exp9_v3/COVID19'
#: Fid score : 21.83055898962805
#syn_images='/content/drive/MyDrive/Colab Notebooks/cxr_v1_exp2/exp2_200epoch'  : Fid score : 22.911621421695592
#v5_sys_images='/content/drive/MyDrive/Colab Notebooks/cxr_v1_exp5/200_epoch' 374.44

In [ ]:
syn_images_exp2_v3_names=os.listdir(syn_images_exp2_v3)
print(syn_images_exp2_v3_names[1:10])

syn_images_exp4_v3_names=os.listdir(syn_images_exp4_v3)
print(syn_images_exp4_v3_names[1:10])

syn_images_exp9_v3_names=os.listdir(syn_images_exp9_v3)
print(syn_images_exp9_v3_names[1:10])

['cxr1_exp4_v4_gen_img_1499_1.png', 'cxr1_exp4_v4_gen_img_1499_2.png', 'cxr1_exp4_v4_gen_img_1499_3.png', 'cxr1_exp4_v4_gen_img_1499_4.png', 'cxr1_exp4_v4_gen_img_1499_5.png', 'cxr1_exp4_v4_gen_img_1499_6.png', 'cxr1_exp4_v4_gen_img_1499_7.png', 'cxr1_exp4_v4_gen_img_1499_8.png', 'cxr1_exp4_v4_gen_img_1499_9.png']
['cxr1_exp4_v3_gen_img_1499_1.png', 'cxr1_exp4_v3_gen_img_1499_2.png', 'cxr1_exp4_v3_gen_img_1499_3.png', 'cxr1_exp4_v3_gen_img_1499_4.png', 'cxr1_exp4_v3_gen_img_1499_5.png', 'cxr1_exp4_v3_gen_img_1499_6.png', 'cxr1_exp4_v3_gen_img_1499_7.png', 'cxr1_exp4_v3_gen_img_1499_8.png', 'cxr1_exp4_v3_gen_img_1499_9.png']
['aug_exp9_v3_gen_img_999_1.png', 'aug_exp9_v3_gen_img_999_2.png', 'aug_exp9_v3_gen_img_999_3.png', 'aug_exp9_v3_gen_img_999_4.png', 'aug_exp9_v3_gen_img_999_5.png', 'aug_exp9_v3_gen_img_999_6.png', 'aug_exp9_v3_gen_img_999_7.png', 'aug_exp9_v3_gen_img_999_8.png', 'aug_exp9_v3_gen_img_999_9.png']


In [ ]:
#print("Total images present in the test set :",len(test_covid_names+test_normal_names))
print("Total images present in the Synthetic image set exp2_v3 :",len(syn_images_exp2_v3_names))
print("Total images present in the Synthetic image set exp4_v3:",len(syn_images_exp4_v3_names))
print("Total images present in the Synthetic image set exp9_v3:",len(syn_images_exp9_v3_names))
#print("Total images present in the test set :",len(test_covid_names+test_normal_names))


Total images present in the Synthetic image set exp2_v3 : 1000
Total images present in the Synthetic image set exp4_v3: 1000
Total images present in the Synthetic image set exp9_v3: 1000


In [ ]:
syn_dataset_exp2_v3=keras.preprocessing.image_dataset_from_directory(
    syn_images_exp2_v3, label_mode=None, image_size=(256,256), batch_size=64,shuffle=True
)
syn_dataset_exp2_v3 = syn_dataset_exp2_v3.map(lambda x: x / 255.0)



Found 1000 files belonging to 1 classes.


In [ ]:
syn_dataset_exp4_v3=keras.preprocessing.image_dataset_from_directory(
    syn_images_exp4_v3, label_mode=None, image_size=(256,256), batch_size=64,shuffle=True
)
syn_dataset_exp4_v3 = syn_dataset_exp4_v3.map(lambda x: x / 255.0)



Found 1000 files belonging to 1 classes.


In [ ]:
syn_dataset_exp9_v3=keras.preprocessing.image_dataset_from_directory(
    syn_images_exp9_v3, label_mode=None, image_size=(256,256), batch_size=64,shuffle=True
)
syn_dataset_exp9_v3 = syn_dataset_exp9_v3.map(lambda x: x / 255.0)



Found 1000 files belonging to 1 classes.


In [ ]:
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

In [ ]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [ ]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
def compute_embeddings(dataloader, count):
    image_embeddings = []
    inception_model = tf.keras.applications.InceptionV3(include_top=False, 
                              input_shape=(256,256,3), 
                              pooling='avg')
    for _ in tqdm(range(count)):
        images = next(iter(dataloader))
        embeddings = inception_model.predict(images)

        image_embeddings.extend(embeddings)

    return np.array(image_embeddings)

In [ ]:
import math
count = math.ceil(507/64)
count_syn=math.ceil(1000/64)

In [ ]:
real_image_embeddings = compute_embeddings(real_dataset, count)
exp2_v3 = compute_embeddings(syn_dataset_exp2_v3, count_syn)
exp4_v3 = compute_embeddings(syn_dataset_exp4_v3, count_syn)
exp9_v3 = compute_embeddings(syn_dataset_exp9_v3, count_syn)


100%|██████████| 16/16 [03:13<00:00, 12.10s/it]


In [ ]:
from scipy.linalg import sqrtm
def calculate_fid(real_embeddings, generated_embeddings):
    # calculate mean and covariance statistics
    mu1, sigma1 = real_embeddings.mean(axis=0), np.cov(real_embeddings, rowvar=False)
    mu2, sigma2 = generated_embeddings.mean(axis=0), np.cov(generated_embeddings,  rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
      covmean = covmean.real
     # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [ ]:
fid = calculate_fid(real_image_embeddings, exp2_v3)

In [ ]:
fid

200.6005867910286

In [ ]:
fid1 = calculate_fid(real_image_embeddings, exp4_v3)

In [ ]:
fid1

212.38586207870043

In [ ]:
fid2 = calculate_fid(real_image_embeddings, exp9_v3)

In [ ]:
fid3

292.16584440268053

In [ ]:
def compute_embeddings_new(dataloader, count):
    image_embeddings = []
    inception_model = tf.keras.applications.InceptionV3(include_top=False, 
                              input_shape=(256,256,3), weights="imagenet",
                              pooling='avg')
    for _ in tqdm(range(count)):
        images = next(iter(dataloader))
        embeddings = inception_model.predict(images)

        image_embeddings.extend(embeddings)

    return np.array(image_embeddings)

In [ ]:
real_image_embeddings_new = compute_embeddings_new(real_dataset, count)
exp2_v3_new = compute_embeddings_new(syn_dataset_exp2_v3, count_syn)
exp4_v3_new = compute_embeddings_new(syn_dataset_exp4_v3, count_syn)
exp9_v3_new = compute_embeddings_new(syn_dataset_exp9_v3, count_syn)


100%|██████████| 16/16 [03:13<00:00, 12.10s/it]


In [ ]:
fid_exp2_v3 = calculate_fid(real_image_embeddings_new, exp2_v3_new)
fid1_exp4_v3 = calculate_fid(real_image_embeddings_new, exp4_v3_new)
fid2_exp9_v3 = calculate_fid(real_image_embeddings_new, exp9_v3_new)

In [ ]:
print('FID score for exp2_v3',fid_exp2_v3)
print('FID score for exp4_v3',fid1_exp4_v3)
print('FID score for exp9_v3',fid2_exp9_v3)

FID score for exp2_v3 196.52670349465402
FID score for exp4_v3 207.8615662398231
FID score for exp9_v3 293.2972475239209
